# Декораторы
**Декоратор** - паттерн, который позволяет расширить функцию без изменений ее исходного кода.

Он бывает:
1) сихнронным или асинхронным
2) с параметрами или без
3) с кешированием или без

**Общий принцип**: func = decorator(func)

**Паттерны**:  
func = decorator(func)  
func = decorator(params)(func)  
func = decorator(func(params))

## Синхронный и асинхронный декораторы

### Синхронный декоратор

In [57]:
from typing import Callable, Any
from functools import wraps
from time import sleep
from random import random


def retry(tries: int = 3, delay: float = 1.0) -> Callable:
    '''
    Decorator for re-executing a function in case of an error.
    '''
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapped(*args: Any, **kwargs: Any):
            last_exception = None
            for attempt in range(tries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    print(f'Attempt {attempt + 1}/{tries} failed. Error: "{e}"')
                    if attempt < tries - 1:
                        sleep(delay)
            raise last_exception
        return wrapped
    return decorator


@retry()
def cube(number: int) -> int:
    '''
    Returns the number powered by three.
    '''
    sleep(2)
    if random() < 0.5:
        raise ValueError('Random error during calculation!')
    return number ** 3


def main():
    result = cube(3)
    print(result)


if __name__ == '__main__':
    main()

Attempt 1/3 failed. Error: "Random error during calculation!"
Attempt 2/3 failed. Error: "Random error during calculation!"
27


### Асинхронный декоратор

In [59]:
import asyncio
from typing import Callable, Coroutine, Any
from functools import wraps


def log(coroutine: Coroutine) -> Callable:
    '''
    Decorator for logging an intermediate result.
    '''
    @wraps(coroutine)
    async def wrapper(*args: Any, **kwargs: Any):
        print(f'Starting "{coroutine.__name__}" with args{args} and kwargs{kwargs}...')
        result = await coroutine(*args, **kwargs)
        print(f'Result: {result}')
        return result
    return wrapper


@log
async def cube(number: int) -> int:
    '''
    Returns the number powered by three.
    '''
    await asyncio.sleep(2)
    return number * number * number


async def main():
    await cube(3)
    

# for .py
# asyncio.run(main=main())
# for .ipynb
await main()

Starting "cube" with args(3,) and kwargs{}...
Result: 27


### Кешируемый декоратор

In [2]:
from functools import lru_cache
import time


@lru_cache
def calculations():
    time.sleep(3)
    return True


calculations()
calculations()
calculations()

True